In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GraphConv
from torch_geometric.nn.norm import BatchNorm
import torch.nn as nn
import os
import pandas as pd
import numpy as np
import itertools
from math import radians, sin, cos, sqrt, atan2
import numpy as np
from torch.optim.swa_utils import AveragedModel, SWALR
from sklearn.model_selection import train_test_split


/mnt/HDD/RyanFolder/projects/Weather-Predition-Project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# <div align="center">**ZE EPIC DATA**</div>

In [2]:
# Directory containing the CSV files
directory = '../processed-final-data-2'

# Dictionary to store the dataframes
dataframes = {}

# Iterate over all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Extract the file name without extension and convert it to int
        key = int(os.path.splitext(filename)[0])
        
        # Read the CSV file into a dataframe
        df = pd.read_csv(os.path.join(directory, filename))
        dataframes[key] = df


# Print the dictionary keys to verify
print(dataframes.keys())

dict_keys([72790024141, 72785524114, 72789094197, 72793024233, 72785794129, 72788594266, 72797624217, 72785024157, 72797094240, 72798594276, 72792424223, 72792894263, 72781024243, 72781524237, 72788324220, 72698824219, 72793894274, 74206024207, 72782724110, 72793724222, 72792594227, 72782594239, 72794504205, 72792394225, 72784524163, 72792024227, 72785694176])


In [3]:
# Dictionaries to store the training and testing dataframes
train_dataframes = {}
test_dataframes = {}

# Split each dataframe into training and testing sets
for key, df in dataframes.items():
    train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)
    train_dataframes[key] = train_df
    test_dataframes[key] = test_df
    # Check if the maximum index of the training set is less than the minimum index of the testing set
    if train_df.index.max() < test_df.index.min():
        print(f"{key}: Sequential split verified.")
    else:
        print(f"{key}: Sequential split NOT verified.")

# Print the keys of the training and testing dictionaries to verify
print(train_dataframes.keys())
print(test_dataframes.keys())

72790024141: Sequential split verified.
72785524114: Sequential split verified.
72789094197: Sequential split verified.
72793024233: Sequential split verified.
72785794129: Sequential split verified.
72788594266: Sequential split verified.
72797624217: Sequential split verified.
72785024157: Sequential split verified.
72797094240: Sequential split verified.
72798594276: Sequential split verified.
72792424223: Sequential split verified.
72792894263: Sequential split verified.
72781024243: Sequential split verified.
72781524237: Sequential split verified.
72788324220: Sequential split verified.
72698824219: Sequential split verified.
72793894274: Sequential split verified.
74206024207: Sequential split verified.
72782724110: Sequential split verified.
72793724222: Sequential split verified.
72792594227: Sequential split verified.
72782594239: Sequential split verified.
72794504205: Sequential split verified.
72792394225: Sequential split verified.
72784524163: Sequential split verified.


In [4]:
def create_node_features_sequences(dataframes):
    # Create a list to store the node features for each time step for input and desired output
    node_features_sequence_input = []
    node_features_sequence_output = []

    # Iterate over the rows of the dataframes (assuming all dataframes have the same number of rows)
    for i in range(len(next(iter(dataframes.values())))):
        if i == len(next(iter(dataframes.values()))) - 1:
            break
        # Create a list to store the features of all nodes at the current time step for input
        node_features_input = []
        # Create a list to store the features of all nodes at the next time step for output
        node_features_output = []

        # Iterate over each dataframe and extract the features at the current row for input
        # and the next row for output
        for key, df in dataframes.items():
            node_features_input.append((df.iloc[i].values - df.iloc[i].mean()) / df.iloc[i].std())
            node_features_output.append(df.iloc[i + 1].values)

        # Stack the features of all nodes to create a 2D array (num_nodes, num_features)
        node_features_sequence_input.append(np.stack(node_features_input))
        node_features_sequence_output.append(np.stack(node_features_output))

    # Convert the lists to numpy arrays (time_steps, num_nodes, num_features)
    node_features_sequence_input = np.array(node_features_sequence_input)
    node_features_sequence_output = np.array(node_features_sequence_output)

    # Convert the numpy arrays to torch tensors
    node_features_sequence_input = torch.tensor(node_features_sequence_input, dtype=torch.float)
    node_features_sequence_output = torch.tensor(node_features_sequence_output, dtype=torch.float)

    return node_features_sequence_input, node_features_sequence_output

# Call the function and print the shapes of the resulting tensors
node_features_sequence_input_train, node_features_sequence_output_train = create_node_features_sequences(train_dataframes)
node_features_sequence_input_test, node_features_sequence_output_test = create_node_features_sequences(test_dataframes)
print(node_features_sequence_input_train.shape)
print(node_features_sequence_output_test.shape)

torch.Size([525, 27, 59])
torch.Size([131, 27, 59])


In [5]:
# Calculate mean and standard deviation of the training data
mean = node_features_sequence_input_train.mean(dim=(0, 1), keepdim=True)
std = node_features_sequence_input_train.std(dim=(0, 1), keepdim=True)

# Normalize the training and testing data
# node_features_sequence_input_train = (node_features_sequence_input_train - mean) / std
# node_features_sequence_input_test = (node_features_sequence_input_test - mean) / std

# Print the mean and standard deviation to verify
print("Mean:", mean)
print("Standard Deviation:", std)

Mean: tensor([[[-0.3513, -0.2447, -0.3517,  0.0810,  4.4760,  1.6202,  2.2340,
           2.6113,  2.9742, -0.0595,  1.6904,  1.6897, -0.3517, -0.3467,
          -0.3515, -0.3516, -0.3516, -0.3514, -0.3494, -0.3177, -0.3505,
          -0.3467, -0.3517, -0.3466, -0.3516, -0.3510, -0.3514, -0.3517,
          -0.3517, -0.3515, -0.3515, -0.3502, -0.3516, -0.3516, -0.3465,
          -0.3514, -0.3516, -0.3516, -0.3517, -0.3163, -0.3511, -0.3512,
          -0.3515, -0.3148, -0.3511, -0.3516, -0.3466, -0.3514, -0.3516,
          -0.3512, -0.3365, -0.3515, -0.3514, -0.3513, -0.3513, -0.3516,
          -0.3515, -0.3515, -0.3511]]])
Standard Deviation: tensor([[[0.0163, 0.1418, 0.0161, 0.2685, 1.1921, 0.2288, 0.4864, 0.5617,
          0.8244, 0.1007, 0.2498, 0.2488, 0.0161, 0.0229, 0.0165, 0.0163,
          0.0163, 0.0165, 0.0216, 0.0314, 0.0179, 0.0229, 0.0161, 0.0277,
          0.0163, 0.0172, 0.0166, 0.0162, 0.0161, 0.0165, 0.0164, 0.0197,
          0.0163, 0.0162, 0.0279, 0.0166, 0.0163, 0.01

# <div align="center">**ZE EPIC EDGE DATA**</div>

In [6]:
# Import the location-datamap.csv file as a dataframe
location_datamap_df = pd.read_csv('../location-datamap.csv')

# Print the first few rows of the dataframe to verify
print(location_datamap_df.head())

        STATION  LONGITUDE  LATITUDE  ELEVATION
0  7.279372e+10 -122.28308  47.92322      167.1
1  7.278462e+10 -118.28572  46.09456      356.7
2  7.420712e+10 -122.58333  47.08333       91.4
3  7.279762e+10 -122.54069  48.79910       45.9
4  7.279239e+10 -123.93074  46.97288        4.5


In [7]:
def haversine_distance(lat1, lon1, lat2, lon2, el1=0, el2=0):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c

    # Elevation difference
    height = el2 - el1

    # Calculate the total distance considering elevation
    total_distance = sqrt(distance**2 + height**2)

    return total_distance

In [8]:
# Create a dictionary to store the distances between nodes
distances = {}

# Define the number of nodes
num_nodes = len(dataframes)

# Iterate over each pair of nodes
for i, j in itertools.combinations(range(num_nodes), 2):
    # Get the station IDs for the nodes
    station_i = list(dataframes.keys())[i]
    station_j = list(dataframes.keys())[j]
    
    # Get the location data for the stations
    location_i = location_datamap_df[location_datamap_df['STATION'] == station_i].iloc[0]
    location_j = location_datamap_df[location_datamap_df['STATION'] == station_j].iloc[0]
    
    # Calculate the distance between the stations
    distance = haversine_distance(location_i['LATITUDE'], location_i['LONGITUDE'], location_j['LATITUDE'], location_j['LONGITUDE'], location_i['ELEVATION'], location_j['ELEVATION'])
    
    # Store the distance in the dictionary
    distances[(i, j)] = distance

# Print the distances to verify
print(distances)

{(0, 1): 395.4679457314314, (0, 2): 129.15783117310457, (0, 3): 342.5330553210191, (0, 4): 438.10430559873856, (0, 5): 432.0504921333856, (0, 6): 437.11321302131137, (0, 7): 369.0087247919059, (0, 8): 503.7031148557305, (0, 9): 455.6811328123654, (0, 10): 474.59654599988056, (0, 11): 348.0950737502581, (0, 12): 128.6980383337319, (0, 13): 835.9738346074198, (0, 14): 171.59537540199554, (0, 15): 384.56540064951065, (0, 16): 373.58764051749137, (0, 17): 362.1843030623904, (0, 18): 32.53657446575969, (0, 19): 306.4908073667999, (0, 20): 404.50991220772727, (0, 21): 52.81240627430448, (0, 22): 404.8085057270694, (0, 23): 505.4288197147213, (0, 24): 286.0836462058196, (0, 25): 412.6227567328569, (0, 26): 272.19719043646785, (1, 2): 390.14029117618804, (1, 3): 727.6917541369755, (1, 4): 109.85904921372176, (1, 5): 798.995937782049, (1, 6): 802.4581375032445, (1, 7): 33.690956835057186, (1, 8): 865.1737430163511, (1, 9): 825.6499844120128, (1, 10): 860.6728795779842, (1, 11): 724.996392306694

In [9]:
# Create edge index and edge attributes from distances dictionary
edge_index = []
edge_attr = []

for (i, j), distance in distances.items():
    edge_index.append([i, j])
    edge_index.append([j, i])  # Assuming undirected graph
    edge_attr.append([distance])
    edge_attr.append([distance])  # Assuming undirected graph

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor(edge_attr, dtype=torch.float)

# <div align="center">**ZE EPIC GPU IMPORT**</div>

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Move data to device
node_features_sequence_input_train = node_features_sequence_input_train.to(device)
node_features_sequence_output_test = node_features_sequence_output_test.to(device)
edge_index = edge_index.to(device)
edge_attr = edge_attr.to(device)

# <div align="center">**ZE EPIC BATCHING**</div>

In [12]:
batch_size = node_features_sequence_input_train.shape[0] # Adjust this value based on your GPU memory capacity
# Calculate the number of full batches
num_full_batches_train = (node_features_sequence_input_train.size(0) // batch_size) * batch_size

# Trim the input and output tensors to have a size divisible by the batch size
trimmed_input_train = node_features_sequence_input_train[:num_full_batches_train]
trimmed_output_train = node_features_sequence_output_train[:num_full_batches_train]

# Create batches of data
batched_input_train = trimmed_input_train.view(-1, batch_size, trimmed_input_train.size(1), trimmed_input_train.size(2))
batched_output_train = trimmed_output_train.view(-1, batch_size, trimmed_output_train.size(1), trimmed_output_train.size(2))

# Adjust the number of batches
num_batches_train = batched_input_train.size(0)

# Print the batch size and number of full batches to verify
print("Batch size:", batch_size)
print("Number of full batches:", num_full_batches_train)

Batch size: 525
Number of full batches: 525


In [13]:
batch_size = node_features_sequence_input_test.shape[0]  # Adjust this value based on your GPU memory capacity
# Calculate the number of full batches
num_full_batches_test = (node_features_sequence_input_test.size(0) // batch_size) * batch_size

# Trim the input and output tensors to have a size divisible by the batch size
trimmed_input_test = node_features_sequence_input_test[:num_full_batches_test]
trimmed_output_test = node_features_sequence_output_test[:num_full_batches_test]

# Create batches of data
batched_input_test = trimmed_input_test.view(-1, batch_size, trimmed_input_test.size(1), trimmed_input_test.size(2))
batched_output_test = trimmed_output_test.view(-1, batch_size, trimmed_output_test.size(1), trimmed_output_test.size(2))

# Adjust the number of batches
num_batches_test = batched_input_test.size(0)
print("Batch size:", batch_size)
print("Number of full batches:", num_full_batches_train)

Batch size: 131
Number of full batches: 525


# <div align="center">**ZE EPIC MODEL**</div>

In [14]:
class GNNWithTransformer(torch.nn.Module):
    def __init__(self, in_channels, edge_in_channels, hidden_channels, out_channels, num_transformer_layers):
        super(GNNWithTransformer, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels, heads=4, concat=False)  # GAT layer
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.batch_norm1 = BatchNorm(hidden_channels)
        self.batch_norm2 = BatchNorm(hidden_channels)
        self.batch_norm3 = BatchNorm(hidden_channels)
        self.dropout = torch.nn.Dropout(p=0.5)
        
        # Transformer encoder layer
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_channels, nhead=16),
            num_layers=num_transformer_layers
        )
        
        self.output_layer = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_attr):
        # Compute inverse distance weights
        edge_weights = 1.0 / (edge_attr + 1e-6)  # Adding a small value to avoid division by zero
        
        # Apply the first GCN convolution with edge weights
        x = self.conv1(x, edge_index, edge_weight=edge_weights)
        x = self.batch_norm1(x)
        x = F.sigmoid(x)
        x = self.dropout(x)
        
        # Apply the second GAT convolution with edge weights
        x = self.conv2(x, edge_index, edge_attr=edge_weights)
        x = self.batch_norm2(x)
        x = F.sigmoid(x)
        x = self.dropout(x)

        # Apply the third Graph convolution with edge weights
        x = self.conv3(x, edge_index, edge_weight=edge_weights)
        x = self.batch_norm3(x)
        x = F.sigmoid(x)
        x = self.dropout(x)
        
        # Prepare data for Transformer
        x = x.unsqueeze(0)  # Add batch dimension
        
        # Apply Transformer encoder
        x = self.transformer_encoder(x)
        
        x = x.squeeze(0)  # Remove batch dimension
        
        # Apply the output layer
        x = self.output_layer(x)
        
        return x

# <div align="center">**ZE EPIC MODEL SETUP**</div>

In [15]:
in_channels = node_features_sequence_input_train.shape[2]
out_channels = in_channels  

edge_in_channels = 1
hidden_channels = 256

num_transformer_layers = 8
print("Number of input channels:", in_channels)
print("Number of output channels:", out_channels)
print("Number of hidden channels:", hidden_channels)
print("Number of transformer layers:", num_transformer_layers)

model = GNNWithTransformer(in_channels, edge_in_channels, hidden_channels, out_channels, num_transformer_layers).to(device)

Number of input channels: 59
Number of output channels: 59
Number of hidden channels: 256
Number of transformer layers: 8


/mnt/HDD/RyanFolder/projects/Weather-Predition-Project/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
# Define the parameters
num_epochs = 1000  # Adjust the number of epochs as needed
learning_rate = 0.5
scheduler_mode = 'min'
scheduler_factor = 0.5
scheduler_patience = 5
# num_warmpup_steps = 10 
# num_training_steps = num_epochs * num_batches_train

#Print all parameter 
print("Number of epochs:", num_epochs)
print("Learning rate:", learning_rate)
print("Scheduler mode:", scheduler_mode)
print("Scheduler factor:", scheduler_factor)
print("Scheduler patience:", scheduler_patience)
# print("Number of warmup steps:", num_warmpup_steps)
# print("Number of training steps:", num_training_steps)

# Define the optimizer and loss function
swa_model = AveragedModel(model)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode=scheduler_mode, factor=scheduler_factor, patience=scheduler_patience, verbose=True)
swa_scheduler = SWALR(optimizer, swa_lr=0.05)
loss_fn = torch.nn.MSELoss()

Number of epochs: 1000
Learning rate: 0.5
Scheduler mode: min
Scheduler factor: 0.8
Scheduler patience: 10


/mnt/HDD/RyanFolder/projects/Weather-Predition-Project/.venv/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


# <div align="center">**ZE EPIC MODEL TRAIN**</div>

In [ ]:
# Training loop
epoch_losses = []
epoch_maes = []

# Set the model to training mode
model.train()

for epoch in range(num_epochs):
    total_loss = 0
    total_mae = 0
    
    optimizer.zero_grad()
    
    for b in range(num_batches_train):
        # Get the batched node features and desired output
        node_features_batch = batched_input_train[b].to(device)
        desired_output_batch = batched_output_train[b].to(device)
        
        # Reshape node_features_batch to match the expected input shape for the model
        node_features_batch = node_features_batch.view(-1, node_features_batch.size(2)).to(device)
        
        model_output_batch = model(node_features_batch, edge_index, edge_attr)
        
        # Reshape model_output_batch back to the original shape
        model_output_batch = model_output_batch.view(batched_input_train.size(1), -1, model_output_batch.size(1))
        loss = loss_fn(model_output_batch, desired_output_batch)
        
        # Backward pass
        loss.backward()
        
        # Update the optimizer
        optimizer.step()
        optimizer.zero_grad()
        
        # Compute MAE for debugging
        mae = torch.mean(torch.abs(model_output_batch - desired_output_batch))
        
        total_loss += loss.item()
        total_mae += mae.item()

    
    
    average_loss = total_loss / num_batches_train
    average_mae = total_mae / num_batches_train
    epoch_losses.append(average_loss)
    epoch_maes.append(average_mae)

    scheduler.step(average_loss)

    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch+1}, Average Loss: {average_loss}, Average MAE: {average_mae}, Learning Rate: {current_lr}")

Epoch 1, Average Loss: 248.6512451171875, Average MAE: 5.599358081817627, Learning Rate: 0.5
Epoch 2, Average Loss: 1457.7384033203125, Average MAE: 34.20182418823242, Learning Rate: 0.5
Epoch 3, Average Loss: 27896.666015625, Average MAE: 164.5419464111328, Learning Rate: 0.5
Epoch 4, Average Loss: 1105.972412109375, Average MAE: 30.61089324951172, Learning Rate: 0.5
Epoch 5, Average Loss: 2103.6279296875, Average MAE: 44.71921157836914, Learning Rate: 0.5
Epoch 6, Average Loss: 913.46875, Average MAE: 29.012243270874023, Learning Rate: 0.5
Epoch 7, Average Loss: 2119.80029296875, Average MAE: 35.38002014160156, Learning Rate: 0.5
Epoch 8, Average Loss: 2924.48291015625, Average MAE: 49.33660125732422, Learning Rate: 0.5
Epoch 9, Average Loss: 465.01373291015625, Average MAE: 19.010194778442383, Learning Rate: 0.5
Epoch 10, Average Loss: 1169.4412841796875, Average MAE: 24.54741668701172, Learning Rate: 0.5
Epoch 11, Average Loss: 1851.9539794921875, Average MAE: 32.55799865722656, Le